<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/StructureDistance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --force-reinstall 'sqlalchemy<2.0.0'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.7/613.7 kB 33.6 MB/s eta 0:00:00
  Attempting uninstall: greenlet
    Found existing installation: greenlet 2.0.2
    Uninstalling greenlet-2.0.2:
      Successfully uninstalled greenlet-2.0.2
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.19
    Uninstalling SQLAlchemy-2.0.19:
      Successfully uninstalled SQLAlchemy-2.0.19


In [2]:
import os
!pwd
os.chdir('/content')
# Clone ALIGNN repo to get example folder
if not os.path.exists('graphdot'):
  !git clone https://gitlab.com/knc6/graphdot.git

os.chdir('graphdot')
# Install using setup.py in case pip didn't work
# !python setup.py devel!op
!pip install -e .
#!pip install dgl-cu111 # Colab has cuda 11.1

/content
Cloning into 'graphdot'...
remote: Enumerating objects: 8001, done.
remote: Counting objects: 100% (3928/3928), done.
remote: Compressing objects: 100% (1259/1259), done.
remote: Total 8001 (delta 2613), reused 3923 (delta 2611), pack-reused 4073
Receiving objects: 100% (8001/8001), 3.08 MiB | 3.24 MiB/s, done.
Resolving deltas: 100% (5078/5078), done.
Obtaining file:///content/graphdot
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.1/353.1 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.8/865.8 kB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [3]:
pip install ase

In [4]:
pip install pymatgen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 46.9 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
from ase.build import molecule, bulk
from graphdot import Graph
from graphdot.kernel.molecular import Tang2019MolecularKernel

# build sample molecules
small_title = ['H2O', 'HCl', 'NaCl']
bulk_title = ['NaCl-bulk', 'NaCl-bulk2']
bulk = [
    bulk('NaCl', 'rocksalt', a=5.64),
    bulk('NaCl', 'rocksalt', a=5.66),
]
molecules = [molecule(name) for name in small_title] + bulk

# convert to molecular graphs
graphs = [Graph.from_ase(m) for m in molecules]

# use pre-defined molecular kernel
kernel = Tang2019MolecularKernel(edge_length_scale=0.1)

R = kernel(graphs)

# normalize the similarity matrix
d = np.diag(R)**-0.5
K = np.diag(d).dot(R).dot(np.diag(d))

# note the difference between the NaCl variants
title = small_title + bulk_title
print(pd.DataFrame(K, columns=title, index=title))

/content/graphdot/graphdot/graph/__init__.py:23: UserWarning: Cannot import RDKit, `graph.from_rdkit()` will be unavailable.

  warnings.warn(


                 H2O       HCl      NaCl  NaCl-bulk  NaCl-bulk2
H2O         1.000000  0.072223  0.030734   0.030734    0.030734
HCl         0.072223  1.000000  0.015842   0.015842    0.015842
NaCl        0.030734  0.015842  1.000000   0.023764    0.023764
NaCl-bulk   0.030734  0.015842  0.023764   1.000000    0.803765
NaCl-bulk2  0.030734  0.015842  0.023764   0.803765    1.000000


In [6]:
pip install jarvis-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 8.8 MB/s eta 0:00:00


In [7]:

from jarvis.db.figshare import data
import pandas as pd
from jarvis.core.atoms import Atoms

dft_3d = data('dft_3d')
df=pd.DataFrame(dft_3d)

Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699


100%|██████████| 40.8M/40.8M [00:03<00:00, 10.5MiB/s]


Loading the zipfile...
Loading completed.


In [17]:
si=['JVASP-1002','JVASP-91933','JVASP-25369']#,'JVASP-25368']
mos2=['JVASP-28733','JVASP-28413','JVASP-58505']
ni3al=['JVASP-14971','JVASP-99749','JVASP-11979']
sio2=['JVASP-58349', 'JVASP-34674','JVASP-34656']

In [35]:
def atoms_jid(ids=[],verbose=True):
  get_atoms=[]
  for i in ids:
    atoms=(Atoms.from_dict(df[df['jid']==i].atoms.values[0]))
    if verbose:
        print(i,atoms.composition.reduced_formula,atoms.get_spacegroup)
    ase_atoms=atoms.ase_converter()
    get_atoms.append(ase_atoms)
  return get_atoms

def get_df(ids=[],verbose=True):
    ase_atoms_list = atoms_jid(ids,verbose=verbose)
    graphs = [Graph.from_ase(m) for m in ase_atoms_list]

    # use pre-defined molecular kernel
    kernel = Tang2019MolecularKernel(edge_length_scale=0.1)

    R = kernel(graphs)

    # normalize the similarity matrix
    d = np.diag(R)**-0.5
    K = np.diag(d).dot(R).dot(np.diag(d))

    # note the difference between the NaCl variants

    df_kernel = (pd.DataFrame(K, columns=ids, index=ids))
    return df_kernel

# si_atoms = atoms_jid(si)

In [39]:
get_df(si)

JVASP-1002 Si [227, 'Fd-3m']
JVASP-91933 Si [12, 'C2/m']
JVASP-25369 Si [223, 'Pm-3n']


,JVASP-1002,JVASP-91933,JVASP-25369
JVASP-1002,1.000000,0.233288,0.163287
JVASP-91933,0.233288,1.000000,0.869847
JVASP-25369,0.163287,0.869847,1.000000


In [40]:
get_df(ni3al)

JVASP-14971 AlNi3 [221, 'Pm-3m']
JVASP-99749 AlNi3 [194, 'P6_3/mmc']
JVASP-11979 AlNi3 [225, 'Fm-3m']


,JVASP-14971,JVASP-99749,JVASP-11979
JVASP-14971,1.000000,0.954035,0.711243
JVASP-99749,0.954035,1.000000,0.808739
JVASP-11979,0.711243,0.808739,1.000000


In [41]:
get_df(mos2)

JVASP-28733 MoS2 [194, 'P6_3/mmc']
JVASP-28413 MoS2 [164, 'P-3m1']
JVASP-58505 MoS2 [216, 'F-43m']


,JVASP-28733,JVASP-28413,JVASP-58505
JVASP-28733,1.000000,0.973766,0.881506
JVASP-28413,0.973766,1.000000,0.857583
JVASP-58505,0.881506,0.857583,1.000000


In [42]:
get_df(sio2)

JVASP-58349 SiO2 [152, 'P3_121']
JVASP-34674 SiO2 [20, 'C222_1']
JVASP-34656 SiO2 [92, 'P4_12_12']


,JVASP-58349,JVASP-34674,JVASP-34656
JVASP-58349,1.000000,0.981050,0.930632
JVASP-34674,0.981050,1.000000,0.963703
JVASP-34656,0.930632,0.963703,1.000000


In [50]:
x = get_df(si+sio2+ni3al+mos2)

JVASP-1002 Si [227, 'Fd-3m']
JVASP-91933 Si [12, 'C2/m']
JVASP-25369 Si [223, 'Pm-3n']
JVASP-58349 SiO2 [152, 'P3_121']
JVASP-34674 SiO2 [20, 'C222_1']
JVASP-34656 SiO2 [92, 'P4_12_12']
JVASP-14971 AlNi3 [221, 'Pm-3m']
JVASP-99749 AlNi3 [194, 'P6_3/mmc']
JVASP-11979 AlNi3 [225, 'Fm-3m']
JVASP-28733 MoS2 [194, 'P6_3/mmc']
JVASP-28413 MoS2 [164, 'P-3m1']
JVASP-58505 MoS2 [216, 'F-43m']


In [51]:
x

,JVASP-1002,JVASP-91933,JVASP-25369,JVASP-58349,JVASP-34674,JVASP-34656,JVASP-14971,JVASP-99749,JVASP-11979,JVASP-28733,JVASP-28413,JVASP-58505
JVASP-1002,1.000000,0.233288,0.163287,0.063762,0.068309,0.073951,0.020443,0.024192,0.029055,0.031724,0.029984,0.032827
JVASP-91933,0.233288,1.000000,0.869847,0.372574,0.399232,0.436911,0.115175,0.136780,0.160568,0.176673,0.164690,0.185581
JVASP-25369,0.163287,0.869847,1.000000,0.390146,0.422565,0.459150,0.118733,0.140791,0.164615,0.181053,0.168165,0.191868
JVASP-58349,0.063762,0.372574,0.390146,1.000000,0.981050,0.930632,0.133847,0.158554,0.181572,0.199133,0.184058,0.213564
JVASP-34674,0.068309,0.399232,0.422565,0.981050,1.000000,0.963703,0.143302,0.169617,0.194363,0.213075,0.197096,0.229485
JVASP-34656,0.073951,0.436911,0.459150,0.930632,0.963703,1.000000,0.154231,0.182902,0.209979,0.231148,0.213520,0.247891
JVASP-14971,0.020443,0.115175,0.118733,0.133847,0.143302,0.154231,1.000000,0.954035,0.711243,0.146091,0.136758,0.156315
JVASP-99749,0.024192,0.136780,0.140791,0.158554,0.169617,0.182902,0.954035,1.000000,0.808739,0.173126,0.161741,0.184594
JVASP-11979,0.029055,0.160568,0.164615,0.181572,0.194363,0.209979,0.711243,0.808739,1.000000,0.202329,0.189739,0.213131
JVASP-28733,0.031724,0.176673,0.181053,0.199133,0.213075,0.231148,0.146091,0.173126,0.202329,1.000000,0.973766,0.881506


In [52]:
import seaborn as sns
cm = sns.light_palette("blue", as_cmap=True)
x=x.style.background_gradient(cmap=cm)
display(x)

,JVASP-1002,JVASP-91933,JVASP-25369,JVASP-58349,JVASP-34674,JVASP-34656,JVASP-14971,JVASP-99749,JVASP-11979,JVASP-28733,JVASP-28413,JVASP-58505
JVASP-1002,1.000000,0.233288,0.163287,0.063762,0.068309,0.073951,0.020443,0.024192,0.029055,0.031724,0.029984,0.032827
JVASP-91933,0.233288,1.000000,0.869847,0.372574,0.399232,0.436911,0.115175,0.136780,0.160568,0.176673,0.164690,0.185581
JVASP-25369,0.163287,0.869847,1.000000,0.390146,0.422565,0.459150,0.118733,0.140791,0.164615,0.181053,0.168165,0.191868
JVASP-58349,0.063762,0.372574,0.390146,1.000000,0.981050,0.930632,0.133847,0.158554,0.181572,0.199133,0.184058,0.213564
JVASP-34674,0.068309,0.399232,0.422565,0.981050,1.000000,0.963703,0.143302,0.169617,0.194363,0.213075,0.197096,0.229485
JVASP-34656,0.073951,0.436911,0.459150,0.930632,0.963703,1.000000,0.154231,0.182902,0.209979,0.231148,0.213520,0.247891
JVASP-14971,0.020443,0.115175,0.118733,0.133847,0.143302,0.154231,1.000000,0.954035,0.711243,0.146091,0.136758,0.156315
JVASP-99749,0.024192,0.136780,0.140791,0.158554,0.169617,0.182902,0.954035,1.000000,0.808739,0.173126,0.161741,0.184594
JVASP-11979,0.029055,0.160568,0.164615,0.181572,0.194363,0.209979,0.711243,0.808739,1.000000,0.202329,0.189739,0.213131
JVASP-28733,0.031724,0.176673,0.181053,0.199133,0.213075,0.231148,0.146091,0.173126,0.202329,1.000000,0.973766,0.881506
